
# Λ<sub>c</sub><sup>+</sup> Reconstruction using ROOT TMVA

As described in [introduction](Introduction.md), it is possible to use the TMVA (Toolkit for Multivariate Data Analysis) with ROOT to perform multivariate analyzes with data from high energy physics experiments.
Here is how to use 3 of the TMVA methods:

* linear cuts
* Neural Networks of type Multilayer Perceptron (MLP)
* Boosted Decision Tree (BDT)

In the latest versions of ROOT it is possible to use all functions in python code. This feature is called pyROOT, an example of its use is shown here.

<p>&nbsp;</p>

In [13]:
from ROOT import TMVA, TCut, TFile
from math import floor

In [9]:
outputFile = TFile( "TMVA_output_4_6.root", 'RECREATE' )
TMVA.Tools.Instance()

factory = TMVA.Factory( "TMVAClassification", outputFile,
                       "!V:Color:DrawProgressBar:Transformations=I;D;P;G,D:AnalysisType=Classification" )

In [3]:
factory.SetVerbose()

Ora dichiariamo un DataLoader e aggiungiamo le variabili usando la AddVariable Function

In [4]:
dataloader = TMVA.DataLoader("dataset")

In [5]:
dataloader.AddVariable( "massK0S", 'F' )
dataloader.AddVariable( "CosThetaStar", 'F' )
dataloader.AddVariable( "combinedProtonProb", 'F')
dataloader.AddVariable( "signd0", 'F')
dataloader.AddVariable( "tImpParV0", 'F')
dataloader.AddVariable( "tImpParBach", 'F' )
dataloader.AddVariable( "CtK0S := DecayLengthK0S*0.497/v0P", 'F')
dataloader.AddVariable( "cosPAK0S", 'F' )

#spectator variables

dataloader.AddSpectator( "nSigmaTOFpr","nSigma TOFpr", 'F' )
dataloader.AddSpectator( "nSigmaTPCpr","nSigma TPCpr", 'F' )
dataloader.AddSpectator( "nSigmaTPCpi","nSigma TPCpi", 'F' )
dataloader.AddSpectator( "nSigmaTPCka","nSigma TPCka", 'F' )
dataloader.AddSpectator( "bachelorEta","bachelor Eta", 'F' )
dataloader.AddSpectator( "LcPt","Lc Pt", 'F' )
dataloader.AddSpectator( "bachelorPt", 'F')
dataloader.AddSpectator( "massLc2K0Sp", "mass Lc-->K0S+p", 'F' )
dataloader.AddSpectator( "massLc2Lambdapi",  "mass Lc -->L(1520)+pi", 'F' )
dataloader.AddSpectator( "massLambda", "mass V0 = Lambda", 'F' )
dataloader.AddSpectator( "massLambdaBar", "mass V0 = LambdaBar", 'F' )
dataloader.AddSpectator( "V0positivePt", "V0 positive Pt", 'F' )
dataloader.AddSpectator( "V0negativePt", "V0 negative Pt", 'F' )
dataloader.AddSpectator( "dcaV0pos", "dca V0 positive", 'F' )
dataloader.AddSpectator( "dcaV0neg", "dca V0 negative", 'F' )
dataloader.AddSpectator( "v0Pt", "K0S Pt", 'F' )
dataloader.AddSpectator( "dcaV0", "dca V0", 'F' )
dataloader.AddSpectator( "V0positiveEta", "V0pos Eta", 'F' )


apriamo il dataset contenente signal and background trees

In [6]:
input= TFile.Open( "AnalysisResults.root")

Warning in <TClass::Init>: no dictionary for class AliRDHFCutsLctoV0 is available
Warning in <TClass::Init>: no dictionary for class AliRDHFCuts is available
Warning in <TClass::Init>: no dictionary for class AliAnalysisCuts is available
Warning in <TClass::Init>: no dictionary for class AliVCuts is available
Warning in <TClass::Init>: no dictionary for class AliESDtrackCuts is available
Warning in <TClass::Init>: no dictionary for class AliAODPidHF is available
Warning in <TClass::Init>: no dictionary for class AliAnalysisUtils is available
Warning in <TClass::Init>: no dictionary for class AliEventCuts is available
Warning in <TClass::Init>: no dictionary for class AliNormalizationCounter is available
Warning in <TClass::Init>: no dictionary for class AliCounterCollection is available


In [7]:
# signal and backg trees for training
signal      = input.Get( "treeList_2_25_2_25_Sgn")
background  = input.Get( "treeList_2_25_2_25_Bkg")

global event weights per tree (see below for setting event-wise weights) Diamo al Training e al Test lo stesso tree, questo verrà quindi diviso in due sottounità che serviranno separatamente per il training e il test.

In [8]:

signalWeight     = 1.0
backgroundWeight = 1.0

dataloader.AddSignalTree(signal, signalWeight)
dataloader.AddBackgroundTree(background, backgroundWeight)

DataSetInfo              : [dataset] : Added class "Signal"
                         : Add Tree treeList_2_25_2_25_Sgn of type Signal with 1762028 events
DataSetInfo              : [dataset] : Added class "Background"
                         : Add Tree treeList_2_25_2_25_Bkg of type Background with 1473789 events


aggiungo dei Tagli sull'impulso trasverso per selezionare solo una parte degli eventi presenti nel tree

In [9]:
mycuts = TCut("LcPt<6 && LcPt>4")
mycutb = TCut("LcPt<6 && LcPt>4")

Applicato il taglio sul momento trasverso il numero di eventi da utilizzare nel training e nel test viene deciso come segue:

1) il training e il testing del segnale viene fatto con il valore minimo tra la metà degli eventi rimasti dopo il taglio e 100000

2) il training e il testing del Bakground viene fatto con il valore minimo tra la metà degli eventi rimasti dopo il taglio e 200000

Training e testing vengono ovviamente fatti con eventi diversi selezionati in maniera randomica all'interno del tree


In [10]:
nTrainingEventsSgn = floor(min(signal.GetEntries("LcPt<6 && LcPt>4")*0.5,100000))
nTrainingEventsBkg = floor(min(background.GetEntries("LcPt<6 && LcPt>4")*0.5,200000))
nTestingEventsSgn = floor(min(signal.GetEntries("LcPt<6 && LcPt>4")*0.5,100000))
nTestingEventsBkg = floor(min(background.GetEntries("LcPt<6 && LcPt>4")*0.5,200000))

Usiamo la funzione DataLoader.PrepareTrainingAndTestTree per applicare i cut agli eventi di input

In [11]:
dataloader.PrepareTrainingAndTestTree( mycuts, mycutb,"nTrain_Signal=nTrainingEventsSgn:nTest_Signal=nTestingEventsSgn:nTrain_Background=nTrainingEventsBkg:nTest_Background=nTestingEventsBkg:SplitMode=Random:NormMode=NumEvents:!V")


In [12]:
%%capture cap --no-stderr 
factory.BookMethod( dataloader, TMVA.Types.kCuts, "Cuts","!H:!V:FitMethod=MC:EffSel:VarProp=FSmart" )
#factory.BookMethod( dataloader, TMVA.Types.kBDT, "BDT","!H:!V:NTrees=850:MinNodeSize=2.5%:MaxDepth=3:BoostType=AdaBoost:AdaBoostBeta=0.5:UseBaggedBoost:BaggedSampleFraction=0.5:SeparationType=GiniIndex:nCuts=20" )
#factory.BookMethod(dataloader, TMVA.Types.kMLP, "MLP","H:!V:NeuronType=tanh:VarTransform=N:NCycles=600:HiddenLayers=N+5:TestRate=5:!UseRegulator" )

# Train MVAs
#factory.TrainAllMethods()
# Test MVAs
#factory.TestAllMethods()
# Evaluate MVAs
#factory.EvaluateAllMethods() 
# Save the output.
outputFile.Close()

Factory                  : Booking method: Cuts
                         : 
                         : Use optimization method: "Monte Carlo"
                         : Use efficiency computation method: "Event Selection"
                         : Use "FSmart" cuts for variable: 'massK0S'
                         : Use "FSmart" cuts for variable: 'CosThetaStar'
                         : Use "FSmart" cuts for variable: 'combinedProtonProb'
                         : Use "FSmart" cuts for variable: 'signd0'
                         : Use "FSmart" cuts for variable: 'tImpParV0'
                         : Use "FSmart" cuts for variable: 'tImpParBach'
                         : Use "FSmart" cuts for variable: 'CtK0S'
                         : Use "FSmart" cuts for variable: 'cosPAK0S'


In [13]:
with open('std_output.txt', 'w') as f:
     f.write(cap.stdout)